<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/version2/modeling/where_modeling/WI_IA_Similarity_PCA_SupetSetData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal

This colab is for cosine similarity 
* to find the counties from Iwoa, that are similar to Winsconsin widespread counties


# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import warnings; warnings.simplefilter('ignore')

In [ ]:
#drive.flush_and_unmount(600000)

In [ ]:
!ls '/content/drive/Shareddrives/CMPE 295- Master Project/projectdata-2021'

 All_CovidDataFeb20.csv
 alldatasets_Feb20.csv
 All_GoogleMobilityDataFeb20.csv
 All_SocialDistancingDataFeb20.csv
 AllStates_Superset_dataset_Feb20.csv
 AllStates_Superset_dataset_Feb21.csv
 CA_alldatasets_Feb05.csv
 CA_alldatasets_Jan13.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb05.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb06.csv
 CA_CombinedData_Labeled_CDCGuideline_Jan15.csv
 CA_combined_Feb04.csv
 CA_CovidDataJan10.csv
 CA_dataset_Feb11.csv
 CA_GoogleMobilityDataJan10.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb05.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb06.csv
 CA_MarylandData_Labeled_CDCGuideline_Jan15.csv
 CA_processedMarylandData_Feb04.csv
 CA_processedMarylandData_Feb05.csv
 CA_processedMarylandData_Feb12.csv
 CA_processedMarylandData_Jan11.csv
 CA_SocialDistancingDataJan10.csv
 CA_socialDistancingInertiaData_Feb04.csv
 CA_SuperData_Labeled_CDCGuideline_Feb13.csv
 CA_SuperData_Labeled_Guideline_Feb26.csv
 CA_Superset_bins_dataset_Feb20.csv
 CA_Superset_dataset_Feb12

# Import package

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler as ss
import itertools
from scipy import linalg
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
all_states_data = pd.read_csv(location+'AllStates_Superset_dataset_Mar26.csv', low_memory=False)
all_states_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k
0,2020-03-01,Alabama,1001,0,Autauga County,0.0,7.194,49,15.0,13.0,0.0,0.0,3.0,0.0,35.853419,Alabama,35,22,2.81,41.6,1.0,32.9,0.26,2.55,0,55601,-1.2,0.11,19,58786,19.0,2.8,48.68,19,101,3.79,0.33,0.025,0.0,0.0,37,47.1,0.0,0.0,0.0,0,0,0.0,28,0.4,3.5,3.2,0.3,0.0,0.0,0.0
1,2020-03-02,Alabama,1001,0,Autauga County,0.0,14.587,100,15.0,4.0,0.0,0.0,5.0,-1.0,35.853419,Alabama,11,13,3.33,40.9,0.5,42.0,0.80,2.53,0,55601,-1.9,0.11,19,58786,19.0,2.8,48.68,19,101,3.79,0.33,0.025,0.0,0.0,38,47.1,0.0,0.0,0.0,0,0,0.0,28,0.4,3.5,3.2,0.3,0.0,0.0,0.0


In [ ]:
base_state= "Wisconsin"
similar_state= "Iowa"

# Load Oregon Binning Superset Data

In [ ]:
or_bin_superset_data = pd.read_csv('/content/drive/Shareddrives/CMPE 295- Master Project/projectdata-2021/Iowa_Superset_bins_dataset_Mar31.csv',low_memory=False)


In [ ]:
print(or_bin_superset_data.shape)


(1803, 58)


In [ ]:
or_bin_superset_data[or_bin_superset_data.isna().any(axis=1)]

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,State,CTNAME


In [ ]:
print(or_bin_superset_data['County Name'].nunique())
or_bin_superset_data['County Name'].unique()

92


array(['Butler County', 'Calhoun County', 'Cherokee County',
       'Clarke County', 'Clay County', 'Dallas County', 'Fayette County',
       'Franklin County', 'Greene County', 'Henry County',
       'Jackson County', 'Jefferson County', 'Lee County',
       'Madison County', 'Marion County', 'Marshall County',
       'Monroe County', 'Montgomery County', 'Shelby County',
       'Washington County', 'Benton County', 'Boone County',
       'Carroll County', 'Crawford County', 'Howard County',
       'Johnson County', 'Polk County', 'Scott County', 'Union County',
       'Humboldt County', 'Fremont County', 'Hamilton County',
       'Taylor County', 'Clayton County', 'Decatur County',
       'Floyd County', 'Jasper County', 'Jones County', 'Mitchell County',
       'Worth County', 'Cass County', 'Clinton County', 'Grundy County',
       'Hancock County', 'Warren County', 'Winnebago County',
       'Delaware County', 'Harrison County', 'Adair County',
       'Allamakee County', 'Appanoos

In [ ]:
or_bin_superset_data['positivity_rate'] = (or_bin_superset_data['New.cases.1000.people'] / or_bin_superset_data['Tests.done.1000.people'])* 100/1000
or_bin_superset_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,State,CTNAME,positivity_rate
0,2020-02-29,Iowa,19023.0,0.000000,Butler County,0.0,16.919500,99.583333,0.0,2.416667,0.0,0.0,1.666667,0.0,9.806636,Iowa,14.833333,16.250000,3.477500,41.891667,0.725000,63.033333,0.563333,2.913333,0.000000,14539.0,11.983333,0.21,29.0,55999.0,0.5,1.3,50.09,6.0,143.0,3.69,0.26,0.011,0.00000,0.001667,44.0,30.000000,8.566667,0.010833,0.000000,0.000000,0.0,0.024167,30.333333,0.800000,3.7,6.100000,0.5,0.000000,0.000000,7.682443,IA,Butler County,0.000000
1,2020-03-15,Iowa,19023.0,0.066667,Butler County,0.0,6.985467,40.800000,0.0,-8.066667,0.0,0.0,-17.400000,0.0,9.806636,Iowa,38.266667,24.866667,2.718667,40.153333,0.946667,45.080000,0.418000,2.300000,0.066667,14539.0,-11.573333,0.21,29.0,55999.0,0.5,1.3,50.09,6.0,143.0,3.69,0.26,0.011,0.00458,0.045333,41.8,30.206399,11.093333,0.721333,0.512667,4.066667,0.4,0.032667,44.800000,16.793333,6.9,23.326667,0.5,0.963333,0.458537,0.262021,IA,Butler County,0.000635


#  Load CA Superset Labeled Data

In [ ]:
ca_labeled_superset_data = all_states_data[all_states_data['Province_State'] == base_state]
ca_labeled_superset_data.head(3)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k
44512,2020-03-01,Wisconsin,55053,0,Jackson County,0.0,2.500,30,22.0,0.0,0.0,23.0,11.0,0.0,8.019932,Wisconsin,27,19,3.27,45.0,7.6,40.7,0.34,2.94,0,20478,3.2,0.13,26,52984,2.1,3.3,53.42,9,120,3.01,0.27,0.045,0.0,0.0,37,41.3,0.0,0.0,0.0,0,0,0.0,7,0.9,3.4,4.5,0.0,0.0,0.0,41.305451
44513,2020-03-02,Wisconsin,55053,0,Jackson County,0.0,8.198,99,0.0,4.0,0.0,14.0,4.0,0.0,8.019932,Wisconsin,19,18,3.44,40.1,6.1,51.1,0.53,2.91,0,20478,2.1,0.13,26,52984,2.1,3.3,53.42,9,120,3.01,0.27,0.045,0.0,0.0,38,41.3,0.0,0.0,0.0,0,0,0.0,7,0.9,3.4,4.5,0.0,0.0,0.0,35.831235
44514,2020-03-03,Wisconsin,55053,0,Jackson County,0.0,5.540,67,11.0,6.0,0.0,22.0,3.0,0.0,8.019932,Wisconsin,16,18,3.76,40.0,6.8,57.8,0.50,3.26,0,20478,14.4,0.13,26,52984,2.1,3.3,53.42,9,120,3.01,0.27,0.045,0.0,0.0,39,41.3,0.0,0.0,0.0,0,0,0.0,14,0.9,3.4,4.5,0.0,0.0,0.0,27.371082


In [ ]:
ca_labeled_superset_data[ca_labeled_superset_data.isna().any(axis=1)]

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k


In [ ]:
ca_labeled_superset_data['Province_State'].unique()

array(['Wisconsin'], dtype=object)

In [ ]:
ca_labeled_superset_data.shape

(17862, 56)

In [ ]:
#ca_labeled_superset_data = ca_labeled_superset_data[ca_labeled_superset_data['growth_label']=='widespread']

In [ ]:
ca_labeled_superset_data.shape

(17862, 56)

In [ ]:
print("CA labeled:", ca_labeled_superset_data.Date.min(),ca_labeled_superset_data.Date.max())
print("Oregon Bining:", or_bin_superset_data.Date.min(),or_bin_superset_data.Date.max())

CA labeled: 2020-03-01 2020-12-28
Oregon Bining: 2020-02-29 2020-12-15


# Filtering widespread data from CA

In [ ]:
#ca_widespread_data = ca_labeled_superset_data[ca_labeled_superset_data['growth_label']=='widespread']
#print(ca_widespread_data.shape)
#ca_widespread_data.head(2)

# Using PCA Columns

## Column Mapping

In [ ]:
ca_labeled_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'Social.distancing.index', 'X..staying.home', 'Trips.person',
       'X..out.of.county.trips', 'X..out.of.state.trips', 'Miles.person',
       'Work.trips.person', 'Non.work.trips.person', 'New.COVID.cases',
       'Population', 'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.people',
       'ICUs.1000.people', 'X..contact.tr

'Province_State', 'FIPS', 'County Name','PopDensity', 'State_Name','State', 'CTFIPS', 'CTNAME', 'STFIPS',
'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density','Testing.capacity', 'Ventilator.shortage','Unemployment.rate','Unemployment.claims.1000.people',

In [ ]:
or_bin_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'Social.distancing.index', 'X..staying.home', 'Trips.person',
       'X..out.of.county.trips', 'X..out.of.state.trips', 'Miles.person',
       'Work.trips.person', 'Non.work.trips.person', 'New.COVID.cases',
       'Population', 'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.people',
       'ICUs.1000.people', 'X..contact.tr

In [ ]:
required_cols = [ 'mask_rule_severity', 'm50_index', #'FIPS',
        'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity',
        'Social.distancing.index','X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person',#'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male','Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.people',
       'ICUs.1000.people','X.days..decreasing.COVID.cases', 'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage', 'Imported.COVID.cases',
       'COVID.exposure.1000.people', 'X.days..decreasing.ILI.cases',
       'Unemployment.claims.1000.people', 'Unemployment.rate',
       'X..working.from.home', 'Cumulative.inflation.rate', 'COVID.death.rate',
       'rolling_avg_new_cases/100k']

### Important columns from PCA

In [ ]:
important_cols = ['Tests.done.1000.people',
 'X..staying.home',
 'Employment.density',
 'workplaces_percent_change_from_baseline',
 'Work.trips.person',
 'X.days..decreasing.ILI.cases',
 'X..out.of.county.trips',
 'grocery_and_pharmacy_percent_change_from_baseline',
 'X..out.of.state.trips',
 'New Cases/100k population',
 'parks_percent_change_from_baseline',
 'm50_index',
 'transit_stations_percent_change_from_baseline',
 'Miles.person',
 'X..Hispanic.Americans',
 'Imported.COVID.cases',
 'retail_and_recreation_percent_change_from_baseline',
 'Transit.mode.share',
 'X..Male',
 'residential_percent_change_from_baseline',
 'X..working.from.home',
 'Testing.capacity']


In [ ]:
if len(important_cols) > len(set(important_cols)):
   print('not unique')


**Summary**: The latent features (loc2vec features ) describe the probalistic causal behaviour of the pandemic. The features are responsible for describing the sudden increase or surge of the pandemic. The counties with closest 

* Surge in LA -> Surge in WA county in OR


In [ ]:
ca_labeled_superset_data_filtered = ca_labeled_superset_data[required_cols]


In [ ]:
or_bin_superset_data_filtered = or_bin_superset_data[required_cols]

#### Updated to select PCA important columns 

In [ ]:

ca_labeled_superset_data_filtered = ca_labeled_superset_data[important_cols]
or_bin_superset_data_filtered = or_bin_superset_data[important_cols]

In [ ]:
print(ca_labeled_superset_data_filtered.shape)
print(or_bin_superset_data_filtered.shape)

(17862, 22)
(1803, 22)


# KNN Similarity between CA data and OR binning reduced data



In [ ]:
#https://towardsdatascience.com/cosine-similarity-how-does-it-measure-the-similarity-maths-behind-and-usage-in-python-50ad30aad7db
#### Reference Code 
# using sklearn to calculate cosine similarity
#from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
#cos_sim=cosine_similarity(A.reshape(1,-1),B.reshape(1,-1))
#print (f"Cosine Similarity between A and B:{cos_sim}")- high is good
#print (f"Cosine Distance between A and B:{1-cos_sim}")- distance less is good

In [ ]:
or_bin_superset_data_filtered.describe().T

,count,mean,std,min,25%,50%,75%,max
Tests.done.1000.people,1803.0,154.665617,124.198264,0.003750,21.314545,136.659167,269.918750,366.597143
X..staying.home,1803.0,22.045303,4.496397,11.500000,18.900000,21.666667,24.800000,40.466667
Employment.density,1803.0,28.179146,59.839710,3.000000,7.000000,11.000000,22.000000,485.000000
workplaces_percent_change_from_baseline,1803.0,-19.762974,10.048257,-50.400000,-26.639610,-20.437500,-13.292614,25.000000
Work.trips.person,1803.0,0.542151,0.128012,0.240000,0.448062,0.526000,0.624476,1.059000
X.days..decreasing.ILI.cases,1803.0,40.440151,35.329456,1.000000,11.200000,28.000000,70.466667,121.333333
X..out.of.county.trips,1803.0,37.396905,8.322145,16.980000,31.686667,37.446667,42.385000,65.650000
Work.trips.person,1803.0,0.542151,0.128012,0.240000,0.448062,0.526000,0.624476,1.059000
grocery_and_pharmacy_percent_change_from_baseline,1803.0,2.520724,8.240509,-29.133333,0.000000,0.000000,0.000000,64.214286
FIPS,1803.0,19098.033833,55.886563,19001.000000,19050.000000,19097.000000,19147.000000,19197.000000


In [ ]:
ca_labeled_superset_data_filtered.describe().T

,count,mean,std,min,25%,50%,75%,max
Tests.done.1000.people,17862.0,257.567040,272.567428,0.000000,19.8500,137.330000,458.350000,905.770000
X..staying.home,17862.0,21.542492,5.393622,8.000000,18.0000,21.000000,24.000000,52.000000
Employment.density,17862.0,56.365581,89.061402,4.000000,10.0000,19.000000,54.000000,447.000000
workplaces_percent_change_from_baseline,17862.0,-21.615944,15.580382,-90.000000,-31.0000,-23.000000,-11.000000,50.000000
Work.trips.person,17862.0,0.448127,0.145469,0.110000,0.3300,0.460000,0.550000,0.940000
X.days..decreasing.ILI.cases,17862.0,43.880696,36.714518,0.000000,14.0000,35.000000,63.000000,133.000000
X..out.of.county.trips,17862.0,35.552827,8.532044,6.800000,30.0000,36.100000,40.800000,71.100000
Work.trips.person,17862.0,0.448127,0.145469,0.110000,0.3300,0.460000,0.550000,0.940000
grocery_and_pharmacy_percent_change_from_baseline,17862.0,2.270238,13.584714,-70.000000,-1.0000,0.000000,6.000000,110.000000
FIPS,17862.0,55072.555705,41.065994,55001.000000,55035.0000,55073.000000,55107.000000,55141.000000


In [ ]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from tqdm import tqdm

def key_consine_similarity(tupple):
    return tupple[1]

# This method computes the distance between each vectors with predicted_vectors input
# in our case, vectors is CA data , predicted_vectors is OR data

def get_computed_similarities(vectors, predicted_vectors, reverse=False):
    data_size = len(vectors) # size of your CA data
    #print("len of OR:", len(predicted_vectors)) #size of OR
    #print("len of CA:", data_size)
    cosine_similarities = []
    simiarity_ids = []
    cosine_distances = []

    for i in range(len(predicted_vectors)): #OR
      max_cosine = 0
      max_j = -1
      #print('actual : ',predicted_vectors[i],' reshaped : ',predicted_vectors[i].reshape(1,-1))
      for j in range(data_size): #CA
        ##cosine_sim_val = (1 - cosine(vectors[j], predicted_vectors[i]))
        # if i == 31:
        #   print(vectors[j].reshape(1,-1))
        #   print(predicted_vectors[i].reshape(1,-1))
        cosine_sim_val = cosine_similarity(vectors[j].reshape(1,-1), predicted_vectors[i].reshape(1,-1)) 
        if max_cosine < cosine_sim_val:
            max_cosine = cosine_sim_val
            max_j = j
      cosine_similarities.append((max_j, max_cosine))
      simiarity_ids.append(max_j)
      cosine_distances.append(max_cosine)
        #if cosine_sim_val > 0.5: # update threshold
          #continue
      #if i > 50: break
        
    #print('len cosine_similarities : ',len(cosine_similarities))
    #return sorted(cosine_similarities, key=key_consine_similarity, reverse=reverse)
    return simiarity_ids,cosine_distances,cosine_similarities

def display_top_n(sorted_cosine_similarities, n=10):
    for i in range(n):
        index, consine_sim_val = sorted_cosine_similarities[i]
        print('News Title: ', merged_data.iloc[index, 0])  
        print('News statement: ', merged_data.iloc[index, 1])  
        print('News label: ', merged_data.iloc[index, 4])  

        print('Cosine Sim Val :', consine_sim_val)
        print('---------------------------------')

In [ ]:
#or_bin_superset_data_filtered.fillna(0)


In [ ]:
or_bin_superset_data_filtered.values[31].reshape(1,-1)

array([[229.56636364,  17.72727273,   5.        ,  -8.        ,
          0.57363636,  22.27272727,  38.        ,   0.        ,
          1.02727273,  28.11911256,   0.        , 103.18181818,
          0.        ,  51.45454545,   1.9       ,  94.        ,
          0.        ,   0.42      ,  51.57      ,   0.        ,
         32.17272727,  10.88181818]])

In [ ]:
%%time
simiarity_ids,cosine_distances,cosine_similarities = get_computed_similarities(vectors=ca_labeled_superset_data_filtered.values, predicted_vectors=or_bin_superset_data_filtered.values)

CPU times: user 1h 44min 29s, sys: 1min 23s, total: 1h 45min 53s
Wall time: 1h 43min 38s


In [ ]:
print(cosine_similarities[:10][0][1])
float(cosine_similarities[:10][0][1])

[[0.99189133]]


0.9918913315271121

In [ ]:
or_bin_superset_data['FIPS'].unique()

array([19023., 19025., 19035., 19039., 19041., 19049., 19065., 19069.,
       19073., 19087., 19097., 19101., 19111., 19121., 19125., 19127.,
       19135., 19137., 19165., 19183., 19011., 19015., 19027., 19047.,
       19089., 19103., 19153., 19163., 19175., 19091., 19071., 19079.,
       19173., 19043., 19053., 19067., 19099., 19105., 19131., 19195.,
       19029., 19045., 19075., 19081., 19181., 19189., 19055., 19085.,
       19001., 19005., 19007., 19013., 19017., 19019., 19021., 19031.,
       19033., 19037., 19051., 19057., 19059., 19061., 19063., 19077.,
       19083., 19093., 19095., 19107., 19109., 19113., 19115., 19117.,
       19123., 19129., 19133., 19139., 19141., 19145., 19147., 19149.,
       19151., 19155., 19157., 19161., 19167., 19169., 19171., 19179.,
       19187., 19191., 19193., 19197.])

In [ ]:
or_bin_superset_data['similiary_score'] = cosine_similarities


# Extracting cosine distance value

In [ ]:
cosine_distance_array = []
for cosine_distance in cosine_distances:
  #print(cosine_distance[0][0])
  cosine_distance_array.append(cosine_distance[0][0])
  


or_bin_superset_data['cosine_distance'] = cosine_distance_array


# Assigning Similar counties from similarity Id

In [ ]:
#print("CA row number=",or_bin_superset_data.iloc[0]['similiary_score'][0])
similar_county_names = []
for id in simiarity_ids:
  similar_county_name = ca_labeled_superset_data.iloc[id]['County Name']
  similar_county_names.append(similar_county_name)

print('length: ',len(similar_county_names))
or_bin_superset_data['similar_county']= similar_county_names
or_bin_superset_data.head(2)

length:  1803


,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,State,CTNAME,positivity_rate,similiary_score,cosine_distance,similar_county
0,2020-02-29,Iowa,19023.0,0.000000,Butler County,0.0,16.919500,99.583333,0.0,2.416667,0.0,0.0,1.666667,0.0,9.806636,Iowa,14.833333,16.250000,3.477500,41.891667,0.725000,63.033333,0.563333,2.913333,0.000000,14539.0,11.983333,0.21,29.0,55999.0,0.5,1.3,50.09,6.0,143.0,3.69,0.26,0.011,0.00000,0.001667,44.0,30.000000,8.566667,0.010833,0.000000,0.000000,0.0,0.024167,30.333333,0.800000,3.7,6.100000,0.5,0.000000,0.000000,7.682443,IA,Butler County,0.000000,"(1140, [[0.9918913315271121]])",0.991891,Clark County
1,2020-03-15,Iowa,19023.0,0.066667,Butler County,0.0,6.985467,40.800000,0.0,-8.066667,0.0,0.0,-17.400000,0.0,9.806636,Iowa,38.266667,24.866667,2.718667,40.153333,0.946667,45.080000,0.418000,2.300000,0.066667,14539.0,-11.573333,0.21,29.0,55999.0,0.5,1.3,50.09,6.0,143.0,3.69,0.26,0.011,0.00458,0.045333,41.8,30.206399,11.093333,0.721333,0.512667,4.066667,0.4,0.032667,44.800000,16.793333,6.9,23.326667,0.5,0.963333,0.458537,0.262021,IA,Butler County,0.000635,"(3617, [[0.9789300336887169]])",0.978930,Taylor County


In [ ]:
or_bin_superset_data.to_csv("IA_testdata_labeled.csv",index=False)

In [ ]:
or_bin_superset_data.tail()

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,State,CTNAME,positivity_rate,similiary_score,cosine_distance,similar_county
1798,2020-10-15,Iowa,19197.0,7.357143,Wright County,0.0,7.534786,136.285714,0.0,0.0,0.0,0.0,-11.142857,0.0,8.517437,Iowa,21.642857,22.785714,3.867143,37.492857,1.435714,69.014286,0.641429,3.227857,7.357143,12690.0,34.342857,0.11,30.0,50198.0,0.7,11.6,49.85,9.0,157.0,3.69,0.26,0.011,0.504707,9.702143,1.785714,36.792133,12.535714,270.097857,15.384286,47.642857,127.428571,24.347857,8.000000,1.500000,3.600000,31.835714,1.600000,2.102143,57.975909,44.145318,IA,Wright County,0.000187,"(1887, [[0.9864887706709642]])",0.986489,Crawford County
1799,2020-10-31,Iowa,19197.0,16.461538,Wright County,0.0,5.830308,105.307692,0.0,0.0,0.0,0.0,-10.076923,0.0,8.517437,Iowa,24.692308,26.846154,3.825385,34.492308,0.884615,59.915385,0.628462,3.199231,16.461538,12690.0,33.138462,0.11,30.0,50198.0,0.7,11.6,49.85,9.0,157.0,3.69,0.26,0.011,1.214369,17.771538,0.153846,42.262586,14.953846,295.876154,23.686923,78.000000,207.076923,33.921538,2.692308,1.869231,3.600000,31.369231,1.692308,1.770769,129.720555,118.636289,IA,Wright County,0.000410,"(8627, [[0.9903359611732967]])",0.990336,Barron County
1800,2020-11-15,Iowa,19197.0,16.666667,Wright County,0.0,6.267333,113.333333,0.0,0.0,0.0,0.0,-20.250000,0.0,8.517437,Iowa,31.083333,28.916667,3.143333,39.766667,1.416667,55.291667,0.624167,2.518333,16.666667,12690.0,4.791667,0.11,30.0,50198.0,0.7,11.6,49.85,9.0,157.0,3.69,0.26,0.011,1.311192,30.340833,1.083333,50.213760,18.016667,326.600000,33.210000,132.666667,352.333333,47.740000,3.500000,2.400000,4.250000,32.450000,1.700000,1.466667,131.337011,142.594469,IA,Wright County,0.000401,"(10001, [[0.9951338847813583]])",0.995134,Dunn County
1801,2020-11-30,Iowa,19197.0,14.200000,Wright County,0.0,5.641900,102.000000,0.0,0.0,0.0,0.0,-12.800000,0.0,8.517437,Iowa,23.800000,25.400000,3.862000,33.510000,1.310000,52.520000,0.639000,3.222000,14.200000,12690.0,34.090000,0.11,30.0,50198.0,0.7,11.6,49.85,9.0,157.0,3.69,0.26,0.011,1.061200,30.797000,4.400000,44.122496,19.300000,345.744000,25.616000,133.900000,358.100000,59.601000,15.400000,1.800000,4.960000,33.800000,1.790000,1.403000,111.899133,114.938647,IA,Wright County,0.000307,"(10001, [[0.9974277486158915]])",0.997428,Dunn County
1802,2020-12-15,Iowa,19197.0,6.333333,Wright County,0.0,5.065889,91.333333,0.0,0.0,0.0,0.0,-32.111111,0.0,8.517437,Iowa,29.222222,27.111111,3.137778,39.866667,1.588889,71.622222,0.582222,2.557778,6.333333,12690.0,6.455556,0.11,30.0,50198.0,0.7,11.6,49.85,9.0,157.0,3.69,0.26,0.011,0.475722,17.207778,1.666667,39.378401,20.166667,366.353333,15.505556,96.222222,195.333333,67.407778,28.000000,1.800000,5.766667,31.322222,1.800000,1.733333,49.908064,66.043754,IA,Wright County,0.000130,"(7313, [[0.9932238263584161]])",0.9

In [ ]:
print(ca_labeled_superset_data_filtered.shape)
ca_labeled_superset_data_filtered.head(2)

(17862, 22)


,Tests.done.1000.people,X..staying.home,Employment.density,workplaces_percent_change_from_baseline,Work.trips.person,X.days..decreasing.ILI.cases,X..out.of.county.trips,grocery_and_pharmacy_percent_change_from_baseline,X..out.of.state.trips,New Cases/100k population,parks_percent_change_from_baseline,m50_index,transit_stations_percent_change_from_baseline,Miles.person,X..Hispanic.Americans,Imported.COVID.cases,retail_and_recreation_percent_change_from_baseline,Transit.mode.share,X..Male,residential_percent_change_from_baseline,X..working.from.home,Testing.capacity
44512,0.0,19,9,11.0,0.34,7,45.0,0.0,7.6,0.0,0.0,30,23.0,40.7,3.3,0,22.0,0.13,53.42,0.0,4.5,0.0
44513,0.0,18,9,4.0,0.53,7,40.1,4.0,6.1,0.0,0.0,99,14.0,51.1,3.3,0,0.0,0.13,53.42,0.0,4.5,0.0


In [ ]:
or_bin_superset_data.head(4)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,State,CTNAME,positivity_rate,similiary_score,similar_county,cosine_distance
0,2020-02-29,Oregon,41029.0,0.142857,Jackson County,0.0,4.668500,90.071429,6.857143,7.714286,43.214286,-13.357143,0.857143,-0.357143,29.723811,Oregon,20.357143,20.428571,3.827143,10.035714,2.635714,36.807143,0.591429,3.236429,0.142857,219564.0,0.642857,1.11,29.0,50851.0,0.6,12.5,48.75,32.0,158.0,2.2,0.2,0.024,0.000643,0.002143,17.000000,48.812683,5.992857,0.035714,0.025714,0.214286,0.000000,0.061429,0.0000,1.000000,3.300000,7.250000,0.600000,0.000000,0.065064,9.063012,OR,Jackson County,0.001800,"(2823, [[0.9883481068672355]])",Humboldt County,0.988348
1,2020-03-15,Oregon,41029.0,1.125000,Jackson County,0.0,2.250875,43.125000,-31.625000,-4.500000,29.687500,-30.750000,-29.187500,9.375000,29.723811,Oregon,41.562500,29.000000,3.193750,9.775000,2.700000,27.906250,0.477500,2.715625,1.125000,219564.0,-15.550000,1.11,29.0,50851.0,0.6,12.5,48.75,32.0,158.0,2.2,0.2,0.024,0.003894,0.055000,3.375000,49.306294,5.418750,1.125000,1.244375,9.562500,1.937500,0.046250,3.0625,8.468750,6.643750,23.131250,0.600000,3.675000,0.512379,0.203325,OR,Jackson County,0.000346,"(11924, [[0.9766964669771497]])",Shasta County,0.976696
2,2020-03-31,Oregon,41029.0,1.800000,Jackson County,0.0,1.390867,26.466667,-38.800000,-11.333333,25.866667,-31.666667,-39.133333,12.466667,29.723811,Oregon,47.066667,30.400000,3.115333,9.020000,2.333333,24.760000,0.438000,2.679333,1.800000,219564.0,-16.680000,1.11,29.0,50851.0,0.6,12.5,48.75,32.0,158.0,2.2,0.2,0.024,0.009413,0.264000,2.466667,50.132514,5.160000,5.416667,5.640667,25.933333,9.866667,0.060667,18.2000,11.873333,13.086667,26.306667,0.226667,12.996000,0.819807,1.071387,OR,Jackson County,0.000174,"(11952, [[0.9804524258485321]])",Shasta County,0.980452
3,2020-04-15,Oregon,41029.0,0.142857,Jackson County,0.0,1.776214,33.857143,-30.428571,-8.000000,42.285714,-30.142857,-36.000000,10.142857,29.723811,Oregon,38.857143,26.785714,3.280714,10.100000,2.585714,27.792857,0.425714,2.855714,0.214286,219564.0,-11.192857,1.11,29.0,50851.0,0.6,12.5,48.75,32.0,158.0,2.2,0.2,0.024,0.000964,0.420714,9.142857,50.062779,4.814286,10.334286,8.497143,25.785714,19.571429,0.092857,33.5000,10.242857,14.614286,28.185714,0.200000,20.927857,0.065064,0.139423,OR,Jackson County,0.000009,"(11950, [[0.9818341067852269]])",Shasta County,0.981834


### Save Temp File to CSV [For Analysis]

In [ ]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'IA_SuperData_With_SimilarityScore_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
or_bin_superset_data.to_csv(location+filename,index=False)

IA_SuperData_With_SimilarityScore_Mar31.csv
